### Representación eje-ángulo de una matriz de rotación

Autor: José M. Buenaposada (josemiguel.buenaposada@urjc.es)

Curso: 2022-2023

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

In [2]:
def rot_matrix_to_axis_angle(R):
    """
      Función que calcula la representación eje de rotación (3D) y ángulo de giro, 
      para una matriz de rotación dada.
    """
    
    # Calcular la descomposición espectral de R (diagonalización)
    eigval, eigvec = np.linalg.eig(R)
#    print(eigval)
#    print(eigvec)
    
    # eigenvector with eigenvalue == 1 is the rotation axis
    index_axis = np.where(eigval==1)[0][0]
    rot_axis = eigvec[:, index_axis]
    
    # trace(R) = 1 + 2 * cos(angle)
    rot_angle = np.arccos((np.trace(R) - 1) / 2)
    
    return np.float64(rot_axis), rot_angle

In [3]:
def axis_angle_to_rot_matrix(rot_axis, rot_angle):
    """
      Función que calcula la matriz de rotación a partir de la 
      representación eje de rotación (3D) y ángulo de giro, 
    """
    
    rot_axis /= np.linalg.norm(rot_axis)
    
    cross = np.array([[0, -rot_axis[2], rot_axis[1]], 
                      [rot_axis[2], 0, -rot_axis[0]],
                      [-rot_axis[1], rot_axis[0], 0]])
    
    rot_axis = rot_axis[:, np.newaxis]
    U = rot_axis @ rot_axis.T
    R = np.cos(rot_angle) * np.eye(3) + np.sin(rot_angle) * cross + (1 - np.cos(rot_angle)) * U
    
    return R

In [4]:
# Rotación alrededor del eje  X de 45 grados
angle_x = np.deg2rad(45)
Rx = np.array([[1, 0, 0], 
               [0, np.cos(angle_x), -np.sin(angle_x)], 
               [0, np.sin(angle_x), np.cos(angle_x)]])

# Rotación alrededor del eje Y de 35 grados
angle_y = np.deg2rad(35)
Ry = np.array([[np.cos(angle_y), 0, np.sin(angle_y)], 
               [0, 1, 0],
               [-np.sin(angle_y), 0, np.cos(angle_y)]])

R = Ry

In [5]:
rot_axis, rot_angle = rot_matrix_to_axis_angle(R)

print(rot_axis.shape)

print("The rotation axis of R is =\n", np.float64(rot_axis))
print("and the angle is =", np.rad2deg(rot_angle))


(3,)
The rotation axis of R is =
 [0. 1. 0.]
and the angle is = 34.99999999999999


/var/folders/bg/skm_74295j948bkpn_fg4hp40000gn/T/ipykernel_11613/368503972.py:19: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.float64(rot_axis), rot_angle


In [6]:
# Recuperación de la matriz de rotación a partir de la representación eje-ángulo.
Rnew = axis_angle_to_rot_matrix(rot_axis, rot_angle)

print("The original rotation matrix is =\n", R)
print("The recovered rotation matrix is =\n", Rnew)


The original rotation matrix is =
 [[ 0.81915204  0.          0.57357644]
 [ 0.          1.          0.        ]
 [-0.57357644  0.          0.81915204]]
The recovered rotation matrix is =
 [[ 0.81915204  0.          0.57357644]
 [ 0.          1.          0.        ]
 [-0.57357644  0.          0.81915204]]


In [7]:
# Using the cv2.Rodrigues method.
vector_rodrigues, _ = cv2.Rodrigues(R)
rot_angle_rodrigues = np.linalg.norm(vector_rodrigues)
rot_axis_rodrigues = vector_rodrigues / rot_angle_rodrigues

print("The rotation axis of R is (cv2.Rodrigues) =\n", rot_axis_rodrigues)
print("and the angle is (cv2.Rodrigues) =", np.rad2deg(rot_angle_rodrigues))

The rotation axis of R is (cv2.Rodrigues) =
 [[0.]
 [1.]
 [0.]]
and the angle is (cv2.Rodrigues) = 34.99999999999999
